In [1]:
import pandas as pd
import csv
import pyltr
import numpy as np
import datetime
import re
import time
import dawg
from nltk import ngrams
from heapq import nlargest
from operator import itemgetter

with open('../data/ngram_dict.csv', mode='r') as infile:
    reader = csv.reader(infile)
#     ngram_dict = {rows[0]:rows[1] for rows in reader}
    ngram_data=[tuple([line[0], int(line[1])]) for line in reader if int(line[1])>2]
ngram_dict = dawg.IntCompletionDAWG(ngram_data)
# prefix_suffix_pairs_background = pd.read_csv("../data/prefix_suffix_pairs.txt")

# Importing all the necessary dictionaries
suffixes = pd.read_csv("../data/Freq_background.csv", index_col='Unnamed: 0')
with open('../data/sorted_popular_queries.csv', mode='r') as infile:
    reader = csv.reader(infile)
    skipheader = next(reader)
    data=[tuple([line[1], int(line[2])]) for line in reader if int(line[2])>2]
sortedpopulardict = dawg.IntCompletionDAWG(data)


data = []
ngram_data = []

top100k = suffixes.iloc[range(100000),:]
suffixes = []
top100ktuple = [tuple(x) for x in top100k.values]
# top100ktuple = [tuple([line["0"], int(line["counts"])])]
top100kdict = dawg.IntCompletionDAWG(top100ktuple)
top100k = []

/root/miniconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:

def get_single_ngram_features(query_input, ngram_dict):
    ngram_scores = [0,0,0,0,0,0]

    for n in range(1,7):
        
        sixgrams = ngrams(query_input.split(), n)
        for grams in sixgrams:
            try:
                ngram_scores[n-1] = ngram_scores[n-1] + ngram_dict.get(' '.join(grams),0)
            except:
                pass
    return ngram_scores

def get_ngram_features(inputs, ngram_dict):
    results = [[get_single_ngram_features(x, ngram_dict)] for x in inputs];
    return results


In [3]:
# Read in training data
removed_session_training = pd.read_csv('../data/removed_session_training.csv', header=None, index_col=[0])
removed_session_training[1] = removed_session_training[1].str.replace('[\W_]+', ' ')

In [4]:
## Process training
training_queries = removed_session_training.reset_index(drop=True)
candidate_prefixes = []
right_query = []

amount_of_suffix_splits = 5
for j in range(len(training_queries)):
    current_query = str(training_queries[1][j])
    split_query = current_query.split(" ")
    suffix = ' '.join(split_query[1:])
    prefix = split_query[0] + " "
    
    for i in np.unique(np.linspace(0, len(suffix), amount_of_suffix_splits).astype(int)):
        candidate_prefixes.append(prefix + suffix[:i])
        right_query.append(current_query)
    
print("done")
candidate_prefixes

done


['www ',
 'www bon',
 'www bonsai ',
 'www bonsai wbff',
 'www bonsai wbff org',
 'loislaw ',
 'loislaw c',
 'loislaw co',
 'loislaw com',
 'www ',
 'www tabi',
 'www tabiecum',
 'www tabiecumming',
 'www tabiecummings com',
 'idx ',
 'idx tec',
 'idx techso',
 'idx techsolsc',
 'idx techsolsc com',
 'www ',
 'www brid',
 'www bridlean',
 'www bridleandbit',
 'www bridleandbit com',
 'gall ',
 'gall s',
 'gall sto',
 'gall ston',
 'gall stones',
 'gallstones ',
 'http ',
 'http www flickr com p',
 'http www flickr com photos 88145967 n',
 'http www flickr com photos 88145967 n00 24368586 in p',
 'http www flickr com photos 88145967 n00 24368586 in pool 32148876 n00',
 'href ',
 'href a href alt a http ww',
 'href a href alt a http www flickr com photos 8',
 'href a href alt a http www flickr com photos 88145967 n00 24368586',
 'href a href alt a http www flickr com photos 88145967 n00 24368586 in pool 32148876 n00',
 'http ',
 'http www flickr com p',
 'http www flickr com photos 88145

In [5]:
# Run this to clear memory
all_candidates = []
relevant_candidate = []
qids =  []
ngram_features =  []

In [ ]:
all_candidates = []
relevant_candidate = []
qids =  []
ngram_features =  []

time_taken_top100k = 0
split_time = 0 
appendtime = 0
ngramtime = 0
real_query_time = 0
total_time_begin = time.time()
checkpoint_time = time.time()

for candidate_prefix_i in range(len(candidate_prefixes)):
    if candidate_prefix_i % 10000 == 0:
        end_checkpoint = time.time() - checkpoint_time
        print("Time between checkpoints: " + str(end_checkpoint))
        print("Total time elapsed: " + str(time.time() - total_time_begin))
        print(str(candidate_prefix_i) + '/' + str(len(candidate_prefixes)))
        checkpoint_time = time.time()
    
    candidate_prefix = candidate_prefixes[candidate_prefix_i]
    original_query = right_query[candidate_prefix_i]
    
#     Getting the endterm for every prefix
    split_time_start = time.time()
    splitted = re.split('[\W_]+',candidate_prefix)
    if candidate_prefix[len(candidate_prefix)-1] == ' ':
        endterm = splitted[len(splitted)-2] + " "
    else:
        endterm = splitted[len(splitted)-1]
    split_time = split_time + time.time() - split_time_start
    
    # Creating the synthetic candidates
    start = time.time()
    top10suffix = nlargest(10, top100kdict.items(prefix=candidate_prefix), key=itemgetter(1))
    out = [i[0] for i in top10suffix]    
    preprefix = candidate_prefix[:-len(endterm)]
    outcombined1 = [preprefix + s for s in out]
    time_taken_top100k = time_taken_top100k + time.time() - start
    
    current_candidates = outcombined1
    
    # Getting the top # real queries
    realquerystart = time.time()
    if not(candidate_prefix == "www " or candidate_prefix == "http "):
        top50queries = nlargest(25, sortedpopulardict.items(prefix=candidate_prefix), key=itemgetter(1))
        keys_pop = [i[0] for i in top50queries]
        current_candidates.extend(keys_pop)
    real_query_time = real_query_time + time.time() - realquerystart
    
    # Appending
    append_time_start = time.time()
    all_candidates.extend(current_candidates)    
    relevant_candidate.extend([s == original_query for s in current_candidates])
    qids.extend(np.ones(len(current_candidates)) * candidate_prefix_i)
    appendtime = appendtime + time.time() - append_time_start

    # Creating features
    ngram_time_start = time.time()
    ngram_features.extend(get_ngram_features(current_candidates, ngram_dict))
    ngramtime = ngramtime + time.time() - ngram_time_start
    
total_time = time.time() - total_time_begin

print("Total time: " + str(total_time))
print("Top100k suffix time: " + str(time_taken_top100k))
print("Total split time: " + str(split_time))
print("Total append time: " + str(appendtime))
print("Total ngram time: " + str(ngramtime))
print("Total real query time: " + str(real_query_time))


Time between checkpoints: 0.00042366981506347656
Total time elapsed: 0.0005340576171875
0/12741070
Time between checkpoints: 3.2366862297058105
Total time elapsed: 3.2373695373535156
10000/12741070
Time between checkpoints: 2.0084726810455322
Total time elapsed: 5.246094226837158
20000/12741070
Time between checkpoints: 2.6194863319396973
Total time elapsed: 7.865686416625977
30000/12741070
Time between checkpoints: 2.198982000350952
Total time elapsed: 10.064761877059937
40000/12741070
Time between checkpoints: 2.0540883541107178
Total time elapsed: 12.119237184524536
50000/12741070
Time between checkpoints: 2.272439956665039
Total time elapsed: 14.391811609268188
60000/12741070
Time between checkpoints: 1.8404314517974854
Total time elapsed: 16.232491970062256
70000/12741070
Time between checkpoints: 2.2903153896331787
Total time elapsed: 18.52306294441223
80000/12741070
Time between checkpoints: 1.7624235153198242
Total time elapsed: 20.285631895065308
90000/12741070
Time between ch

Time between checkpoints: 2.2287240028381348
Total time elapsed: 169.02692437171936
830000/12741070
Time between checkpoints: 2.246424674987793
Total time elapsed: 171.27346897125244
840000/12741070
Time between checkpoints: 2.0596158504486084
Total time elapsed: 173.33336806297302
850000/12741070
Time between checkpoints: 1.96488356590271
Total time elapsed: 175.29835677146912
860000/12741070
Time between checkpoints: 2.2315454483032227
Total time elapsed: 177.53015184402466
870000/12741070
Time between checkpoints: 2.0783519744873047
Total time elapsed: 179.6086070537567
880000/12741070
Time between checkpoints: 2.088653087615967
Total time elapsed: 181.69736313819885
890000/12741070
Time between checkpoints: 2.1819422245025635
Total time elapsed: 183.8795657157898
900000/12741070
Time between checkpoints: 2.0830435752868652
Total time elapsed: 185.96270394325256
910000/12741070
Time between checkpoints: 2.167505979537964
Total time elapsed: 188.13030886650085
920000/12741070
Time be

38.144774436950684

In [ ]:
# query = input("Query: ")
# original_query = input("Original query: ")

# search_query = re.sub('[\W_]+',' ', query)
# original_query = re.sub('[\W_]+',' ', original_query)

# startt = time.time()
# splitted = re.split('[\W_]+',search_query)
# print(splitted)
# if query[len(query)-1] == ' ':
#     endterm = splitted[len(splitted)-2] + " "
# else:
#     endterm = splitted[len(splitted)-1]

# print("Endterm -> '" + endterm + "'")

# out = top100k[top100k['0'].str.startswith(endterm)].nlargest(10, 'counts')
# out2 = sortedpopular[sortedpopular['0'].str.startswith(search_query)].nlargest(40, 'counts')

# outcombined1 = query[:-len(endterm)] + out['0'].astype(str)
# time.time() - startt
# all_candidates = outcombined1.append(out2.iloc[:,0])
# relevant_candidate = (all_candidates == original_query).apply(float)

# train_input = pd.DataFrame({
#     'query': all_candidates,
#     'relevant': relevant_candidate,
#     'qid': 1
# })
  
# train_input['ngram_features'] = get_ngram_features(all_candidates, ngram_dict)
# print(train_input)


In [ ]:
metric = pyltr.metrics.err.ERR(highest_score=1, gain_type='identity')

model = pyltr.models.LambdaMART(
    metric=metric,
    verbose=1,
)

TX2 = ngram_features.reshape(-1, 6) 
Ty2 = np.array(relevant_candidate)
Tqids2 = np.array(qids)
model.fit(TX2, Ty2, Tqids2)

In [ ]:
with open('../data/train.txt') as trainfile, \
        open('../data/vali.txt') as valifile, \
        open('../data/test.txt') as evalfile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

In [ ]:
EX = TX2 
Ey = Ty2
Eqids = Tqids2

Epred = model.predict(EX)

In [ ]:
metric_MRR = MRR.MRR(highest_score=1, gain_type='identity', k=10)

print('Random ranking:' + str(metric_MRR.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric_MRR.calc_mean(Eqids, Ey, Epred)))

In [ ]:
import MRR

In [ ]:
sorted_pred_y = pyltr.util.sort.get_sorted_y(Ey, Epred)
evaluateMRR(Eqids, sorted_pred_y)

In [ ]:
import pyltr.metrics.err
pyltr.metrics.err.ERR.evaluate = evaluate(self, qid, targets):
    residual = 1.0
    result = 0.0
    for i, t in enumerate(targets[:self.k]):
        assert t <= self.highest_score
        sprob = self._get_satisfied_prob(t)
        result += residual * sprob / (1.0 + i)
        residual *= (1.0 - sprob)
        if residual < _EPS:
            break
    return result

In [ ]:
metric = pyltr.metrics.NDCG(gain_type='identity', k=10)
print('Our model:' + str(metric.evaluate_preds(Eqids, Ey, Epred)))

inputs = ['hello kitty', 'hello my name is jeff', 'hello i am martijn', 'hello who are you', 'hello goodbye', 'hello kitty', 'hello my name is jeff', 'hello i am martijn', 'hello who are you', 'hello goodbye']
EX = get_ngram_features(inputs,ngram_dict)
EX = np.append([], EX)
EX = EX.reshape(-1, 6) 
Ey = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
Eqids = [0,0,0,0,0,1,1,1,1,1]

Epred = model.predict(EX)
print(Epred)
print('Random ranking:' + str(metric.calc_mean_random(Eqids, Ey)))
print('Our model:' + str(metric.evaluate_preds(Eqids, Ey, Epred)))
output = pd.DataFrame({
    'inputs': inputs,
    'value': Epred
})
output.sort_values('value', ascending=False)

In [ ]:
Eqids